# Install Libs

In [ ]:
!pip install llama-index-tools-finance -q
!pip install llama-index -q

In [3]:
from llama_index.tools.finance import FinanceAgentToolSpec

The whole code in LlamaIndex repo:

https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-finance/llama_index/tools/finance

https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-finance/llama_index/tools/finance/base.py

In [4]:
FinanceAgentToolSpec

llama_index.tools.finance.base.FinanceAgentToolSpec

# Specify API Keys

In [6]:
from google.colab import userdata
POLYGON_API_KEY = userdata.get('POLYGON_API_KEY')
FINNHUB_API_KEY = userdata.get('FINNHUB_API_KEY')
ALPHA_VANTAGE_API_KEY = userdata.get('ALPHAVANTAGE_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Various Pre-Configured Tools

In [13]:
tool_spec = FinanceAgentToolSpec(
        POLYGON_API_KEY,
        FINNHUB_API_KEY,
        ALPHA_VANTAGE_API_KEY,
        NEWS_API_KEY
    )

In [12]:
tool_spec.spec_functions

['find_similar_companies',
 'get_earnings_history',
 'get_stocks_with_upcoming_earnings',
 'get_current_gainer_stocks',
 'get_current_loser_stocks',
 'get_current_undervalued_growth_stocks',
 'get_current_technology_growth_stocks',
 'get_current_most_traded_stocks',
 'get_current_undervalued_large_cap_stocks',
 'get_current_aggressive_small_cap_stocks',
 'get_trending_finance_news',
 'get_google_trending_searches',
 'get_google_trends_for_query',
 'get_latest_news_for_stock',
 'get_current_stock_price_info']

# Create an Agent:

In [15]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

In [16]:
GPT_MODEL_NAME = "gpt-4o-mini"

def create_agent(
    polygon_api_key: str,
    finnhub_api_key: str,
    alpha_vantage_api_key: str,
    newsapi_api_key: str,
    openai_api_key: str,
) -> OpenAIAgent:
    tool_spec = FinanceAgentToolSpec(
        polygon_api_key,
        finnhub_api_key,
        alpha_vantage_api_key,
        newsapi_api_key,
    )
    llm = OpenAI(temperature=0, model=GPT_MODEL_NAME, api_key=openai_api_key)
    return OpenAIAgent.from_tools(
        tool_spec.to_tool_list(), llm=llm, verbose=True
    )

agent = create_agent(
    POLYGON_API_KEY,
    FINNHUB_API_KEY,
    ALPHA_VANTAGE_API_KEY,
    NEWS_API_KEY,
    OPENAI_API_KEY,
)

# Start Chatting

## Stock Price

In [17]:
response = agent.chat("What was the last closing price of amazon?")

Added user message to memory: What was the last closing price of amazon?
=== Calling Function ===
Calling function: get_current_stock_price_info with args: {"stock_ticker_symbol":"AMZN"}
Got output: {'Current Price': 178.5, 'High Price of the day': 178.9, 'Low Price of the day': 172.6, 'Open Price of the day': 172.74, 'Percentage change': 3.7067}



## Amazon latest news: ==>

This function "get_latest_news_for_stock" returns only NVIDIA news instead of Amazon:

In [18]:
response = agent.chat("What are the latest news about AMZN?")

Added user message to memory: What are the latest news about AMZN?
=== Calling Function ===
Calling function: get_latest_news_for_stock with args: {"stock_id":"AMZN"}
Got output: ['Nvidia stock is sinking. Wall Street analysts say it’s still a buy']



In [25]:
response = agent.chat("I was asking news about Amazon and not NVIDIA.")

Added user message to memory: I was asking news about Amazon and not NVIDIA.
=== Calling Function ===
Calling function: get_latest_news_for_stock with args: {"stock_id":"AMZN"}
Got output: ['Nvidia stock is sinking. Wall Street analysts say it’s still a buy']



In [26]:
response = agent.chat("I was asking news about Amazon and not NVIDIA. What are the latest news about AMZN?")

Added user message to memory: I was asking news about Amazon and not NVIDIA. What are the latest news about AMZN?
=== Calling Function ===
Calling function: get_latest_news_for_stock with args: {"stock_id":"AMZN"}
Got output: ['Nvidia stock is sinking. Wall Street analysts say it’s still a buy']



To modify it: First you need to dig into the code and understand how it was built:

In [29]:
from newsapi import NewsApiClient
from collections import defaultdict

In [30]:
newsapi = NewsApiClient(api_key=NEWS_API_KEY)
cat_to_id = defaultdict(list)
for source in newsapi.get_sources()["sources"]:
    cat_to_id[source["category"]].append(source["id"])
business_sources = [
        "bloomberg",
        "business-insider",
        "financial-post",
        "fortune",
        "info-money",
        "the-wall-street-journal",
    ]

In [56]:
for source in business_sources:
  assert source in cat_to_id["business"]
cat_to_id["business"]

['argaam',
 'australian-financial-review',
 'bloomberg',
 'business-insider',
 'business-insider-uk',
 'die-zeit',
 'financial-post',
 'fortune',
 'handelsblatt',
 'il-sole-24-ore',
 'info-money',
 'les-echos',
 'the-wall-street-journal',
 'wirtschafts-woche']

▶ I modified the param "page" to "page_size" = 3 to get the first 3 articles

▶ If you put in the query only "AMZN" (ticker) ==> You'll get NVIDIA news

▶ If you put Amazon news ==> You'll get Amazon related news

In [57]:
from datetime import datetime, timedelta
start_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

articles = newsapi.get_everything(
        q="AMZN",
        sources=",".join(business_sources),
        from_param=start_date,
        to=end_date,
        language="en",
        sort_by="relevancy",
        page_size=3,
    )["articles"]
articles

[{'source': {'id': 'fortune', 'name': 'Fortune'},
  'author': 'Will Daniel',
  'title': 'Nvidia stock is sinking. Wall Street analysts say it’s still a buy',
  'description': "Despite concerns about margins and guidance, Wall Street is mostly convinced the 'AI revolution' is just getting started, and Nvidia remains a buy.",
  'url': 'https://fortune.com/2024/08/29/nvidia-stock-sinking-wall-street-analysts-still-buy/',
  'urlToImage': 'https://fortune.com/img-assets/wp-content/uploads/2024/08/GettyImages-2155386351-e1724940057818.jpg?resize=1200,600',
  'publishedAt': '2024-08-29T18:34:44Z',
  'content': 'Nvidia turned in yet another stellar earnings report this week, but with the chip giant priced for perfection amid the AI boom, the stock was still down more than 6% Thursday.\xa0The risk when a stock i… [+6630 chars]'}]

In [55]:
from datetime import datetime, timedelta
start_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

articles = newsapi.get_everything(
        q="Amazon news",
        sources=",".join(business_sources),
        from_param=start_date,
        to=end_date,
        language="en",
        sort_by="relevancy",
        page_size=3,
    )["articles"]
articles

[{'source': {'id': 'business-insider', 'name': 'Business Insider'},
  'author': 'Tom Carter',
  'title': 'So, now we know what TV shows Lauren Sánchez and Jeff Bezos binge in their spare time',
  'description': "Lauren Sánchez opened up about her and Jeff Bezos' favorite TV shows in an interview with People magazine.",
  'url': 'https://www.businessinsider.com/jeff-bezos-lauren-sanchez-share-favorite-tv-shows-baby-reindeer-2024-8',
  'urlToImage': 'https://i.insider.com/66d19cdd1d8d2deb96b79669?width=1200&format=jpeg',
  'publishedAt': '2024-08-30T10:49:06Z',
  'content': "Jeff Bezos and Lauren Sánchez.DREW ANGERER/AFP via Getty Images\r\n<ul><li>Lauren Sánchez opened up about her and Jeff Bezos' favorite TV shows in an interview with People magazine.</li><li>Sánchez sai… [+1654 chars]"},
 {'source': {'id': 'business-insider', 'name': 'Business Insider'},
  'author': 'Dan DeFrancesco',
  'title': "The wealthy's new favorite sport is coming for pickleball's crown",
  'description': 'Pad

## Trending News

From seekingalpha

In [62]:
response = agent.chat("What are the top 10 trending news?")

Added user message to memory: What are the top 10 trending news?
=== Calling Function ===
Calling function: get_trending_finance_news with args: {}
Got output: ['Earnings week ahead: Zscaler, DocuSign, NIO, Broadcom and more', 'Catalyst Watch: August Jobs Report, Intel event, Broadcom earnings, and Costco sales', 'Intel resolves R2 litigation', 'Heavily shorted stocks are not always in distress, S3 Partners analysis says', 'Airlines poised to regain altitude after a tough month', 'Electric vehicle adoption: A disappointment or still on track to reach a critical mass?', 'Netflix works to improve reality-TV dubbing to lure more subscribers', 'Macau casino revenue shines in August; dividends come back into play', 'Notable analyst calls this week: Nvidia, Barrick Gold and Dollar General among top picks', "CrowdStrike's results prove the worst is behind it. It's time to buy the stock: HSBC"]



# Eearnings

It provides the earning history of a ticker:

In [31]:
response = agent.chat("What are the earnings of AMZN on 2024?")

Added user message to memory: What are the earnings of AMZN on 2024?
=== Calling Function ===
Calling function: get_earnings_history with args: {"symbol": "AMZN"}
Got output:     Fiscal Date Ending Reported Date Reported EPS Estimated EPS Surprise  \
0           2024-06-30    2024-08-01         1.26          1.03     0.23   
1           2024-03-31    2024-04-30         0.98          0.82     0.16   
2           2023-12-31    2024-02-01            1           0.8      0.2   
3           2023-09-30    2023-10-26         0.94          0.58     0.36   
4           2023-06-30    2023-08-03         0.65          0.35      0.3   
..                 ...           ...          ...           ...      ...   
104         1998-06-30    1998-07-22        -0.07         -0.07        0   
105         1998-03-31    1998-04-27        -0.03         -0.04     0.01   
106         1997-12-31    1998-01-22        -0.03         -0.04     0.01   
107         1997-09-30    1997-10-27        -0.03         -0.03  

It didn't succeed to call the estimated ones "get_latest_earning_estimate"

In [67]:
response = agent.chat("What are the latest actual and estimated earning of AMZN?")

Added user message to memory: What are the latest actual and estimated earning of AMZN?
=== Calling Function ===
Calling function: get_earnings_history with args: {"symbol":"AMZN"}
Got output:     Fiscal Date Ending Reported Date Reported EPS Estimated EPS Surprise  \
0           2024-06-30    2024-08-01         1.26          1.03     0.23   
1           2024-03-31    2024-04-30         0.98          0.82     0.16   
2           2023-12-31    2024-02-01            1           0.8      0.2   
3           2023-09-30    2023-10-26         0.94          0.58     0.36   
4           2023-06-30    2023-08-03         0.65          0.35      0.3   
..                 ...           ...          ...           ...      ...   
104         1998-06-30    1998-07-22        -0.07         -0.07        0   
105         1998-03-31    1998-04-27        -0.03         -0.04     0.01   
106         1997-12-31    1998-01-22        -0.03         -0.04     0.01   
107         1997-09-30    1997-10-27        -0.

# Similar companies

In [34]:
response = agent.chat("What are the similar companies of AMZN?")

Added user message to memory: What are the similar companies of AMZN?
=== Calling Function ===
Calling function: find_similar_companies with args: {"symbol":"AMZN"}
Got output: ['AMZN', 'CPNG', 'EBAY', 'ETSY', 'DDS', 'OLLI', 'M', 'JWN', 'KSS', 'SVV']



## Growth Stocks

"get_current_technology_growth_stocks" didn't succed to get data ==> one needs to modify it ...It uses yfinance url

In [69]:
response = agent.chat("What are the growth stocks in technology sector in US market?")

Added user message to memory: What are the growth stocks in technology sector in US market?
=== Calling Function ===
Calling function: get_current_technology_growth_stocks with args: {}


/usr/local/lib/python3.10/dist-packages/llama_index/tools/finance/util.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(html_clean, header=header)


Got output:                                                    0
0  Will be right back...  Thank you for your pati...



In [55]:
response = agent.chat("What are the US stocks gainers?")

Added user message to memory: What are the US stocks gainers?
=== Calling Function ===
Calling function: get_current_gainer_stocks with args: {}


/usr/local/lib/python3.10/dist-packages/llama_index/tools/finance/util.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(html_clean, header=header)


Got output:                                                    0
0  Will be right back...  Thank you for your pati...



# Key Takeways

◾ Overall, this agent is a good starting point if you want to build one.

◾ You can draw inspiration from the work done here to improve and customize it for your own use case.

◾ You can also use another LLM as AI Agent like Claude 3.5 Sonnet.